In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch

In [2]:
es=Elasticsearch(
    ['http://*******.oraclevcn.com'],
    http_auth=('esadmin', '*********'),
    scheme='http',
    port=9200)

In [3]:
res = es.search(index="psd_course_catalog_cs2hq2", body={"size":10000, "query": {"match_all": {}}}, headers={"SearchUser":"PS"})

In [4]:
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    src = hit["_source"]
    print("ID: {0} Title: {1}".format( src['CRSE_ID_ORCL_ES_ENG'], src['DESCR_ORCL_ES_ENG']))
    print("Description: {0}".format(src['DESCRLONG_ORCL_ES_ENG']))

Got 2868 Hits:
ID: 000042 Title: Data Organisation
Description: This course examines data organisation.
ID: 000044 Title: Special Topic in Ethnomusic
Description: Special Topic in Ethnomusicology
ID: 000050 Title: Electronics 1
Description: This course examines electronic concepts and theory,
ID: 000059 Title: Software Engineering
Description: This course explains the concepts of software engineering.
ID: 000150 Title: Debates in Social Policy
Description: Debates in Social Policy -  First  Year Sociology Subject
ID: 000167 Title: Sociology of work
Description: Sociology of work - Second  Year Sociology Subject
ID: 000170 Title: Social Change & Modernisation
Description: Social Change and Modernisation - Second  Year Sociology Subject
ID: 000098 Title: Human Behaviour 1B
Description: Human Behaviour 1B - Second Year Social Work Subject
ID: 000105 Title: Real Time Instrumentation
Description: This course teaches the concepts of real time instrumentation.
ID: 000077 Title: Industrial Tra

ID: 004334 Title: Intro Art History
Description: 
ID: 002079 Title: Modern Chemistry
Description: 
ID: 002081 Title: Anthropology & Religion
Description: 
ID: 002083 Title: Organizational Behavior
Description: 
ID: 002089 Title: Fundamentals of Speech
Description: 
ID: 002090 Title: Human and Cultural Geog
Description: 
ID: 002096 Title: Major Authors of Amer Lit
Description: 
ID: 001227 Title: Comparative Ethnic Group Relat
Description: 
ID: 001239 Title: Macroeconomics
Description: 
ID: 000263 Title: Advanced Data Networks
Description: This course teaches advanced data networks.
ID: 000264 Title: International Social Develop
Description: International Social Development -  A subject for a Masters of Social Work
ID: 000235 Title: Industrial Relations 1A
Description: This course teaches the introductory concepts of industrial relations: 1A
ID: 001070 Title: Reactoin Kinetics
Description: Molecular descriptoins:  kinetic theory of gases,  models of elementary processes.
ID: 001167 Title

Description: Intermediate Algebra
ID: 667955 Title: Human Biology for Pharmacy
Description: Human Biology for Pharmacy
ID: 667957 Title: Anatomy & Physiology II
Description: Anatomy & Physiology II
ID: 667958 Title: Pharmaceutics & Calculations
Description: Pharmaceutics & Calculations
ID: 667962 Title: Concepts in Pharmacy Tech
Description: Concepts in Pharmacy Tech
ID: 667965 Title: Pharmacy Technician Practicum
Description: Pharmacy Technician Practicum
ID: 667845 Title: Comprehension L2S3EC
Description: Comprehension L2S3EC
ID: 667918 Title: Econometrics II: ARIMA, VAR, a
Description: Econometrics II: ARIMA, VAR, and Cointegration
ID: 667787 Title: Measurement Techniques in Nucl
Description: 
ID: 667657 Title: Voeding 1: Weet wat je eet
Description: 
ID: 667770 Title: Multidisciplinary Biomedical P
Description: 
ID: 667771 Title: Dutch Language Course
Description: 
ID: 667775 Title: Photonics
Description: 
ID: 667776 Title: Knowledge-Based Systems and Ar
Description: 
ID: 667894 Ti

Description: Independent study in musical performance
ID: 001014 Title: Differential Equations
Description: Solutions of first-order differential equations, nth-order linear equations, systems of linear differential equations, series solutions.
ID: 000215 Title: Marxism and Sociology
Description: Marxism & Sociology - - A Paper in a Masters of Arts - Sociology
ID: 000001 Title: Basic Musical Techniques
Description: Basic Music Techniques is a first year subject in the Bachelor of Arts - Music.
ID: 000010 Title: Harmony and Counterpoint 2
Description: Harmony and Counterpoint - Second Year Bachelor of Arts - Music Subject
ID: 000018 Title: The Music of Africa
Description: The Music of Africa - Second Year Bachelor of Arts - Music Subject
ID: 001038 Title: Inrto to Western Civilization
Description: Broad, historical study of major elements in Western heritage.
ID: 001039 Title: Calculus II
Description: Transcendental functions, techniques and applications of integration, indetermininate 